# Cell Optimization

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

import ase.io
from ase.lattice.cubic import FaceCenteredCubic
from ase.build import bulk

from apps.calcexamples.structure_browser import StructureBrowser


import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None
node = False
structures = [("Select a Structure",{"status":False})]

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def refresh_structure_view():
    global viewer, atoms, node
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    if node is False:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

## Step 1: Select Structure

In [ ]:
def on_struct_change(c):
    global atoms, node
    node = struct_browser.results.value
    refresh_structure_view()
    if not node:
        return
    cell_params.value = "Unit cell <br />a = {:.2f} {:.2f} {:.2f} <br />b = {:.2f} {:.2f} {:.2f} <br />c = {:.2f} {:.2f} {:.2f}".format(
        atoms.cell[0][0], atoms.cell[0][1], atoms.cell[0][2],
        atoms.cell[1][0], atoms.cell[1][1], atoms.cell[1][2],
        atoms.cell[2][0], atoms.cell[2][1], atoms.cell[2][2])

    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()
cell_params = ipw.HTML("Cell: ")
clear_output()
display(ipw.VBox([struct_browser, viewer, cell_params]))

## Step 2: Optimize the unit cell

In [ ]:
from aiida.orm import load_node, Code
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.utils import CalculationFactory
from aiida.work.run import submit
from aiida.orm.data.array.kpoints import KpointsData


In [ ]:
def setup_calc():
    kpoints = KpointsData()
    kpoints.set_kpoints_mesh([4,4,4])
    options =  {'max_wallclock_seconds': 3600, 'resources': {'num_machines': 1}, 'queue_name':'normal'}


    if in_codename.value is None:
        print ("Please select a code")
        return None
    if node is False:
        print ("Please select a structure")
        return None
    code = Code.get_from_string(in_codename.value)
    
    parameters = {
        'CONTROL': {
            'calculation': 'vc-relax',
            'restart_mode': 'from_scratch',
            'wf_collect': True,
        },
        'SYSTEM': {
            'ecutwfc': 100.,
            'ecutrho': 400.,
        },
        'ELECTRONS': {
            'scf_must_converge': False,
            'conv_thr': 1.e-6,
        },
    }

    inputs = {
        'code': code,
        'structure': node,
        'pseudo': get_pseudos_from_structure(node, 'SSSP_efficiency_v1.0'),
        'kpoints': kpoints,
        'parameters': ParameterData(dict=parameters),
        'settings': ParameterData(dict={}),
        '_options': options,
    }
    return inputs

In [ ]:
def get_code_options(plugin_classes):
    """
    Return AiiDA codes using a specific set of plugins
    
    :param plugin_classes: a dictionary of the type
      {'pw': 'quantumespresso.pw', 'ph': 'quantumespresso.ph'}
      where the key is a label and the value is the plugin to check for.
      It will return the set of codes that exist on the same machine.
    """
    from aiida.orm.querybuilder import QueryBuilder
    from aiida.orm import Code, Computer
    
    # TODO: add filter to show only computers configured
    # by the current user
    
    qb = QueryBuilder()
    qb.append(Computer,
          filters={'enabled': True},
          project='name', tag='computer')
    ordered_tags = []
    for tag, plugin_class in plugin_classes.iteritems():
        ordered_tags.append(tag)
        qb.append(Code,
                  filters={'attributes.input_plugin': {'==': plugin_class}},
                  project='label',
                  tag='{}code'.format(tag),
                  has_computer='computer'
                 )
    
    all_results = qb.all()
    return [("{}@{}".format(codename, codeset[0]),"{}@{}".format(codename, codeset[0]) )  for codeset in all_results for codename, tag in zip(codeset[1:], ordered_tags)]


in_codename = ipw.Dropdown(options= [('Select a Code', None)] + get_code_options({'pw':'quantumespresso.pw'}))

In [ ]:
PwCalculation = CalculationFactory('quantumespresso.pw')

In [ ]:
def on_click_submit(b):
    with submit_out:
        process = PwCalculation.process()
        inputs = setup_calc()
        if inputs is None:
            pass
        else:
            calculation = submit(process, **inputs)
            print("Submitted calculation N {}".format(calculation.pid))

btn_submit = ipw.Button(description='Submit Cell Opt')
btn_submit.on_click(on_click_submit)
submit_out = ipw.Output()
display(ipw.HBox([in_codename,btn_submit]), submit_out)